In [1]:
#Script intended to correct all sequences in collapsed_files

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain


from os import listdir
from os.path import isfile, join
import os

In [3]:
#General stuff
alphabet = ['A', 'C', 'G', 'T']

In [4]:
sequence = 'NNN'

In [5]:
def find_Ns(sequence):
    '''
        Find all Ns in sequence S
        Output: list of positions where we have N
    '''

    index_list = []

    for i,char in enumerate(sequence):
        if char not in alphabet:
            #print(i)
            index_list.append(i)
    return index_list
def generate_surr_array(index_list, alphabet):
    '''
        Given an index_list, we generate short arrays (length len(index_list)) with
        all combination of letters from alphabet. 
    '''
    L = len(index_list)
    x = []

    while len(x)<len(alphabet)**L:
        if len(x)==0:
            for letter in alphabet:
                x.append(letter)
        else:


            x_2 = []
            for j,element in enumerate(x):
                for letter in alphabet:

                    x_2.append(element + letter)
                    #print(j,element + letter)

            x = x_2
    return x

def fill_Ns(S, x, index_list):
    '''
        Function that returns a list of all possible
        sequences collapsed in S
    '''
    sequence = S

    list_output = []

    for item in x:
        seq_x = sequence
        for i,index in enumerate(index_list):
            seq_x = seq_x[:index]+ item[i]+seq_x[index+1:]
            '''if i == 0:
                seq_x = sequence[:index_list[i]] + item[i]
                #pass
            elif i ==len(index_list)-1:
                seq_x = seq_x + item[i]
                #pass
            else:
                seq_x = seq_x + sequence[index_list[i-1]:i] + item[i]'''
        list_output.append(seq_x)
    return list_output

def uncollapse_N(sequence, alphabet):
    '''
        Given a sequence 'sequence', find all possible sequences created by
        an alphabet 'alphabet' that might be collapsed in 'sequence'
        
        Output:
            -list of all possible sequences giving place to sequence
    '''

    #Find index
    index_Ns = find_Ns(sequence)

    #Create a substitute array
    surr_array = generate_surr_array(index_Ns, alphabet)

    #Replace substitute array in sequence and export
    result = fill_Ns(sequence, surr_array, index_Ns)

    return result

In [6]:
#Read index  
    
    
dir_path = '/home/samuel/Documents/PhD/Quasispecies/Data/'
index_name =  dir_path + 'seqs_index.dict'

index_dict = {}

with open(index_name, 'r') as f:
    for line in f:
        L = line.split('\t')
        index_dict[L[1][:-1]] = int(L[0])

In [8]:
len(list(index_dict.keys()))

1729578

In [11]:
seqs_list = {}
for ikey in index_dict.keys():
    seqs_list[index_dict[ikey]] = ikey

In [16]:
#####Read a step, t
seqs_list = {}
for ikey in index_dict.keys():
    seqs_list[index_dict[ikey]] = ikey
    

WD = '/home/samuel/Documents/PhD/Quasispecies/Data/Sequences_filtered/'
mypath = WD 


#Obtain the file names
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
data_dict = {}
data_dict[43] = {}
data_dict[30] = {}

#Interpret file name, and extract step number from it

for a in onlyfiles: 
    print(f'Processing {a}')
    if a[0] == 'c':
        step = int(a.split('-')[1][1:])
        t=43
        #print(f'step={step} for t={t}')
    elif a[0] == '3':
        step = int(a.split('-')[2])
        t=30
        #print(f'step={step} for t={t}')
    
    file_name = WD  + a
    id_2_nseq = {}
    with open(file_name, 'r') as r:
        for line in r:
            if line[0] == '>':
                abundance = int(line.split('-')[1][:-1])
                #print(abundance)
            else:
                sequence = line[:-1]
                
                hapl = index_dict[sequence]
                id_2_nseq[hapl]= abundance
    
    #data_dict[t][step] = seq_2_ab
    
    
    #id_2_nseq = seq_2_ab
    
    #Search for sequences having Ns in that step
    list_N = []   #list of the id of the sequences
    for id in list(id_2_nseq.keys()):
        seq = seqs_list[id]
        if 'N' in seq:
            list_N.append(id)
            
    new_id_2_nseq = {}    #dict to accumulate the sum of the N sequences

    for sequence_id in list_N:
        sequence = seqs_list[sequence_id]
        #Search all possible chains collapsed
        unc = uncollapse_N(sequence, alphabet)
        #Find those possible chains in terms of dict id
        unc_dict  = {}

        for g in unc:
            try:
                #This is a genotype that we have seen in our experiments
                if index_dict[g] in id_2_nseq.keys():
                    #This is a genotype without N populated in this step, temp
                    unc_dict[index_dict[g]] = id_2_nseq[index_dict[g]] 
                    #Now we have its popuation
            except:
                pass

        normalization = 0

        for n in unc_dict.keys():
            normalization += unc_dict[n]

        for n in unc_dict.keys():
            
            try:
                new_id_2_nseq[n] += int(id_2_nseq[index_dict[sequence]]*unc_dict[n]/normalization)
            except:
                new_id_2_nseq[n] = int(id_2_nseq[index_dict[sequence]]*unc_dict[n]/normalization)

    for n in new_id_2_nseq.keys():
        id_2_nseq[n] += new_id_2_nseq[n]
    
    with open(WD + 'N_rem_rem/' + a.replace('.fasta', '') + '_Nrem_rem.fasta', 'w') as f:
        n_list = list(id_2_nseq.keys())
        n_list.sort()
        i=0
        for n in n_list:
            if n not in list_N:
                
                abundance = id_2_nseq[n]
                sequence = seqs_list[n]
                f.write(f'>{i}-{abundance}\n')
                f.write(f'{sequence}\n')
                i+=1


Processing 30-1-36-EL3_all_all_trim_merged_filter_sort_filter_length_collapsed.fasta
Processing 30-1-28-EL3_all_all_trim_merged_filter_sort_filter_length_collapsed.fasta
Processing c43-p34-3-EL3_all_all_trim_merged_filter_sort_filter_length_collapsed.fasta
Processing c43-p46-3-EL3_all_all_trim_merged_filter_sort_filter_length_collapsed.fasta
Processing 30-1-58-EL3_all_all_trim_merged_filter_sort_filter_length_collapsed.fasta
Processing 30-1-22-EL3_all_all_trim_merged_filter_sort_filter_length_collapsed.fasta
Processing c43-p16-3-EL3_all_all_trim_merged_filter_sort_filter_length_collapsed.fasta
Processing 30-1-40-EL3_all_all_trim_merged_filter_sort_filter_length_collapsed.fasta
Processing c43-p42-3-EL3_all_all_trim_merged_filter_sort_filter_length_collapsed.fasta
Processing 30-1-18-EL3_all_all_trim_merged_filter_sort_filter_length_collapsed.fasta
Processing c43-p12-3-EL3_all_all_trim_merged_filter_sort_filter_length_collapsed.fasta
Processing 30-1-44-EL3_all_all_trim_merged_filter_sort_

In [ ]:


#Find the relative abundance of those chains based on the abundance of the original sequence and the relative abundance of the rest



5421

{2293: 0.0002927037562500861,
 2: 0.9968009201228667,
 1146: 0.0004166724059560049,
 801: 0.0010020799184561771,
 1083: 0.0008264576647061254,
 3238: 0.00021694513698535792,
 2734: 0.00044422099477954243}

In [290]:
new_id_2_nseq

{2: 300273,
 28: 7095,
 33: 5895,
 46: 5558,
 30: 5421,
 64: 3840,
 73: 3817,
 101: 3193,
 129: 2455,
 120: 2228,
 142: 2162,
 22: 2145,
 160: 1717,
 145: 1620,
 181: 1537,
 0: 1319,
 153: 1170,
 12: 926,
 228: 917,
 231: 810,
 362: 771,
 461: 711,
 294: 710,
 239: 705,
 572: 661,
 356: 661,
 396: 657,
 530: 633,
 321: 618,
 477: 493,
 429: 491,
 550: 490,
 413: 484,
 403: 482,
 478: 465,
 470: 461,
 604: 442,
 690: 425,
 1393: 407,
 496: 404,
 624: 393,
 508: 390,
 536: 380,
 614: 366,
 643: 363,
 770: 346,
 485: 344,
 813: 342,
 3103: 320,
 736: 316,
 836: 312,
 996: 301,
 586: 300,
 1054: 293,
 540: 293,
 801: 301,
 600: 285,
 761: 284,
 713: 282,
 571: 280,
 1502: 279,
 620: 259,
 2982: 250,
 3643: 250,
 1083: 248,
 1806: 237,
 1687: 236,
 1010: 236,
 1795: 231,
 409: 229,
 1134: 221,
 909: 215,
 780: 210,
 1565: 206,
 892: 205,
 1228: 204,
 842: 204,
 7086: 204,
 808: 203,
 454: 200,
 1425: 199,
 758: 198,
 843: 197,
 864: 195,
 1177: 194,
 1271: 192,
 1319: 191,
 815: 190,
 943: 

{2: 300273,
 28: 7095,
 33: 5895,
 46: 5558,
 30: 5421,
 64: 3840,
 73: 3817,
 101: 3193,
 129: 2455,
 120: 2228,
 142: 2162,
 22: 2145,
 160: 1717,
 145: 1620,
 181: 1537,
 0: 1319,
 153: 1170,
 12: 926,
 228: 917,
 231: 810,
 362: 771,
 461: 711,
 294: 710,
 239: 705,
 572: 661,
 356: 661,
 396: 657,
 530: 633,
 321: 618,
 477: 493,
 429: 491,
 550: 490,
 413: 484,
 403: 482,
 478: 465,
 470: 461,
 604: 442,
 690: 425,
 1393: 407,
 496: 404,
 624: 393,
 508: 390,
 536: 380,
 614: 366,
 643: 363,
 770: 346,
 485: 344,
 813: 342,
 3103: 320,
 736: 316,
 836: 312,
 996: 301,
 586: 300,
 1054: 293,
 540: 293,
 801: 301,
 600: 285,
 761: 284,
 713: 282,
 571: 280,
 1502: 279,
 620: 259,
 2982: 250,
 3643: 250,
 1083: 248,
 1806: 237,
 1687: 236,
 1010: 236,
 1795: 231,
 409: 229,
 1134: 221,
 909: 215,
 780: 210,
 1565: 206,
 892: 205,
 1228: 204,
 842: 204,
 7086: 204,
 808: 203,
 454: 200,
 1425: 199,
 758: 198,
 843: 197,
 864: 195,
 1177: 194,
 1271: 192,
 1319: 191,
 815: 190,
 943: 